In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("train.csv")
train_data.head()

,Id,difficulty,ciphertext,target
0,ID_88b9bbd73,4,"ob|I K?zzhX*L{83B3Z,FuL*Pusm$83L\t@r$$*38,8s...",10
1,ID_f489bd59f,1,c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1-...,13
2,ID_f90fee9c7,2,1*e4N8$f$0ccOuihkHek$k*V*hoeV$Hj8VhH8...,19
3,ID_8303ced65,1,O8v^10O#to1'#^'^tv1^]s111t01Otaq>-ata_1...,17
4,ID_72abc2cb7,2,eV}H}khfe4b8'S.Vc}{A .#VikV.fV?{$f7$Hjb8...,0


In [11]:
data_by_level = {difficulty: train_data[train_data["difficulty"] == difficulty] for difficulty in train_data["difficulty"].unique()}
print(f"Available difficulties: {data_by_level.keys()}")

Available difficulties: dict_keys([4, 1, 2, 3])


In [8]:
data_by_level[1].head()

,Id,difficulty,ciphertext,target
1,ID_f489bd59f,1,c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1-...,13
3,ID_8303ced65,1,O8v^10O#to1'#^'^tv1^]s111t01Otaq>-ata_1...,17
13,ID_850984d77,1,*#^-G1_#O-#b^'ta8af2%e1|28Oot^12#O-#ys1>c...,18
15,ID_f61f4e2ab,1,*#^-G1\HL\5b#Oz8cA8Ac1|>O-c81 ^vvys1>cX...,6
16,ID_d87237dcc,1,1A]v-O1vT101-OA1#^^fas1-avtOfvT1c...,10


## Difficulty 1

In [51]:
cipher_sample = data_by_level[1]["ciphertext"].tolist()[0]
print(f"Cipher length: {len(cipher_sample)}")
print(cipher_sample)

Cipher length: 300
c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1--#1^]1-a1av1vc]]#ad1]#^-1O1vz#1A'#vva^1#^cd0t1^s1[1o1-^'Ocv1Ov1W881Ov1O1-tO81#Of1A^W11>01av1ad1t#OtA1Wat0s1[1gO8oA^81Wat01v^-1vc__vv1ct1t01t#Ot-tv1ad1'#^zaAA1t0#^cd010#s1[1'vo_0aOt#avt1O#1^t1vOtav]O_t^#11>^-


In [53]:
cipher_bytes = cipher_sample.encode()
for i in range(256):
    shifted_cipher = "".join([chr((c + i)%256) for c in cipher_bytes])
    print(f"{'-'*100}\nShift by: {i}\n{shifted_cipher}\n{'-'*100}")

----------------------------------------------------------------------------------------------------
Shift by: 0
c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1--#1^]1-a1av1vc]]#ad1]#^-1O1vz#1A'#vva^1#^cd0t1^s1[1o1-^'Ocv1Ov1W881Ov1O1-tO81#Of1A^W11>01av1ad1t#OtA1Wat0s1[1gO8oA^81Wat01v^-1vc__vv1ct1t01t#Ot-tv1ad1'#^zaAA1t0#^cd010#s1[1'vo_0aOt#avt1O#1^t1vOtav]O_t^#11>^-
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Shift by: 1
d2}GbBP231P(9pwgpz2X$buwHt2\2t2\202^P.b9p2..$2_^2.b2bw2wd^^$be2^$_.2P2w{$2B($wwb_2$_de1u2_t2\2p2._(Pdw2Pw2X992Pw2P2.uP92$Pg2B_X	22?12bw2be2u$PuB2Xbu1t2\2hP9pB_92Xbu12w_.2wd``ww2du2u12u$Pu.uw2be2($_{bBB2u1$_de121$t2\2(wp`1bPu$bwu2P$2_u2wPubw^P`u_$p	22?_.
---------------------------------------------------------------------------

In [54]:
cipher_bytes = cipher_sample.encode()
for i in range(256):
    shifted_cipher = "".join([chr((c ^ i)) for c in cipher_bytes])
    print(f"{'-'*100}\nShift by: {i}\n{shifted_cipher}\n{'-'*100}")

----------------------------------------------------------------------------------------------------
Shift by: 0
c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1--#1^]1-a1av1vc]]#ad1]#^-1O1vz#1A'#vva^1#^cd0t1^s1[1o1-^'Ocv1Ov1W881Ov1O1-tO81#Of1A^W11>01av1ad1t#OtA1Wat0s1[1gO8oA^81Wat01v^-1vc__vv1ct1t01t#Ot-tv1ad1'#^zaAA1t0#^cd010#s1[1'vo_0aOt#avt1O#1^t1vOtav]O_t^#11>^-
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Shift by: 1
b0}G`@N031N&9nwgnx0V"`uwFr0Z0r0Z0.0\N,`9n0,,"0_\0,`0`w0wb\\"`e0\"_,0N0w{"0@&"ww`_0"_be1u0_r0Z0n0,_&Nbw0Nw0V990Nw0N0,uN90"Ng0@_V	00?10`w0`e0u"Nu@0V`u1r0Z0fN9n@_90V`u10w_,0wb^^ww0bu0u10u"Nu,uw0`e0&"_{`@@0u1"_be101"r0Z0&wn^1`Nu"`wu0N"0_u0wNu`w\N^u_"n	00?_,
---------------------------------------------------------------------------